# Associations of childhood and perinatal blood metals with children’s gut microbiomes in a Canadian gestation cohort
### Shen et al., 2022_Environmental Health Perspectives

### This Jupyter notebook explains the high performance computing workflow of raw sequence data processing

Note: Shotgun metagenome data is large, make sure you have enough space and memory for computing. If possible, use scratch space and later remove intermediate files. All scripts run in unix and python. All participants name is de-identified using "SampleID". All path to file is written as $path.

## Step1: Clean up the raw sequence names

In [ ]:
##cleanup names
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L001_R1_001.fastq.gz "$line"_L001_R1.fastq.gz;done
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L002_R1_001.fastq.gz "$line"_L002_R1.fastq.gz;done
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L001_R2_001.fastq.gz "$line"_L001_R2.fastq.gz;done
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L002_R2_001.fastq.gz "$line"_L002_R2.fastq.gz;done

##merge L001&L002
ls *gz|sort -t_ -k3|uniq|while read line1 && read line2; do cat "$line1" "$line2">../fastqmerge/"$line1"_merge.fastq.gz;done

##cleanup names
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L001_R1.fastq.gz_merge.fastq.gz "$line"_R1.fastq.gz;done
ls *gz|cut -f 1 -d "_" |uniq|while read line; do mv "$line"*_L001_R2.fastq.gz_merge.fastq.gz "$line"_R2.fastq.gz;done

##Check file consistencies after merging and renaming
#Check rename loop difference
if diff /$path/SampleID_DNA_S72_L001_R1_001.fastq.gz \
/$path/SampleID_L001_R1.fastq.gz > /dev/null
then
    echo "No difference"
else
    echo "Difference"
fi

#returns no difference

##Check merge loop difference
# do several mannual merge and see difference
cat /$path/SampleID_L001_R1.fastq.gz \
/$path/SampleID_L002_R1.fastq.gz >test1183_R1.fastq.gz

if diff /$path/SampleID_R1.fastq.gz \
/$path/testSampleID_R1.fastq.gz > /dev/null
then
    echo "No difference"
else
    echo "Difference"
fi
#returns no difference


## Step2: Quality Control raw reads

#### 2.1 Fastqc and multiqc raw reads

In [ ]:
#!/bin/bash
#SBATCH --time=20:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=50G
#SBATCH --job-name=fastqc_geste

cd /$path/fastqmerge
mkdir FastQC_reports_geste
export PATH=$PATH:/mnt/home/shenyike/shotguntest/FastQC/
ls *fastq.gz|while read line; do fastqc -o FastQC_reports_geste/ -t 28 "$line";done

scontrol show job $SLURM_JOB_ID


In [ ]:
multiqc .
#Download multiqc html report to confirm what adapter sequencing center used for the next step

#### 2.2 Trim adapters and start/end reads

In [ ]:
#!/bin/bash
#SBATCH --time=40:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=50G
#SBATCH --job-name=geste_trimmomatic

cd /$path/fastqmerge_trim

ls *gz|cut -f 1 -d "_" |uniq|while read line; do java -jar /opt/software/Trimmomatic/0.39-Java-1.8/trimmomatic-0.39.jar \
PE -phred33 "$line"_R1.fastq.gz "$line"_R2.fastq.gz ../QC_data/"$line"_R1.fastq.gz \
../QC_data/"$line".qcup_R1.fastq.gz ../QC_data/"$line"_R2.fastq.gz ../QC_data/"$line".qcup_R2.fastq.gz \
ILLUMINACLIP:/opt/software/Trimmomatic/0.39-Java-1.8/adapters/NexteraPE-PE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36;done
                                    
scontrol show job $SLURM_JOB_ID


#### 2.3 Fastqc and multiqc trimmed reads & check quality

In [ ]:
#!/bin/bash
#SBATCH --time=20:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=50G
#SBATCH --job-name=fastqc_trimmed_geste

cd /$path/fastqmerge
mkdir FastQC_reports_geste
export PATH=$PATH:/mnt/home/shenyike/shotguntest/FastQC/
ls *fastq.gz|while read line; do fastqc -o FastQC_reports_geste/ -t 28 "$line";done

scontrol show job $SLURM_JOB_ID

In [ ]:
multiqc .
#Download multiqc html report to check the quality of QC

## Step3: Remove host reads (human)

In [ ]:
#human genome download from bowtie (already built)
wget https://genome-idx.s3.amazonaws.com/bt/GRCh38_noalt_as.zip

# One sample, 
# Please note, each sample takes 2.5-4.5 hours to run, it is more practical to submit multiple batch jobs. 
#Below batch script    

#!/bin/bash
#SBATCH --time=5:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=100G
#SBATCH --job-name=bowtie2_SampleID
export PATH=$PATH:/mnt/home/shenyike/miniconda3/bin

cd /$path/metagenome_GESTE2020/GRCh38_noalt_as

bowtie2 -x GRCh38_noalt_as \
-1 /$path/metagenome_GESTE2020/QC_data/SampleID_R1.fastq.gz \
-2 /$path/metagenome_GESTE2020/QC_data/SampleID_R2.fastq.gz \
-S SampleID.sam --threads 28 \
--un-conc-gz /mnt/ls15/scratch/users/shenyike/metagenome_GESTE2021/fastq_dehost/SampleID.dehost.fq.gz

scontrol show job $SLURM_JOB_ID


In [ ]:
#Use the below loop to generate #N batch jobs with different Sample ID and submit all jobs same time

In [ ]:
#mkdir sbatch_folder_dehost #make directory in local computer

import pandas as pd
import numpy as np

name_index = pd.read_excel('File_Contain_Sample_List',header = None)
name_index = np.array(name_index[0])

s = ['#!/bin/bash','#SBATCH --time=5:00:00','#SBATCH --ntasks=1','#SBATCH --cpus-per-task=2',
     '#SBATCH --mem=100G']
for i in name_index:
    with open('sbatch_folder_dehost/'+'dehost_'+str(i)+'.sbatch','w') as file:
        for s_text in s:
            file.write(s_text)
            file.write('\n')
        file.write('#SBATCH --job-name=dehost_'+str(i))
        file.write('\n')
        file.write('\n')
        file.write('\n')
        file.write('export PATH=$PATH:/mnt/home/shenyike/miniconda3/bin')
        file.write('\n')
        file.write('\n')
        file.write('cd /$path/metagenome_GESTE2020/GRCh38_noalt_as')
        file.write('\n')
        file.write('\n')
        cmd = 'bowtie2 -x GRCh38_noalt_as -1 /$path/metagenome_GESTE2020/QC_data/'+str(i)+'_R1.fastq.gz -2 /mnt/ls15/scratch/users/shenyike/metagenome_GESTE2021/QC_data/'+str(i)+'_R2.fastq.gz -S '+str(i)+'.sam --threads 28 --un-conc-gz /mnt/ls15/scratch/users/shenyike/metagenome_GESTE2021/fastq_dehost/'+str(i)+'.dehost.fq.gz'
        file.write(cmd)
        file.write('\n')
        file.write('\n')
        file.write('\n')
        file.write('scontrol show job $SLURM_JOB_ID')

In [ ]:
#HAVE all batches in the same folder
ls *|while read line;do sbatch "$line"; done
#Cleanup names
ls *gz|cut -d "."  -f 1|sort -t. -k1|while read line1; do mv "$line1"*1* "$line1"_dehost_R1.fastq.gz;read line2;mv "$line2"*2*  "$line2"_dehost_R2.fastq.gz;done


## Step 4: Humann3 mapping

In [ ]:
#One sample

#Below batch scripts
#!/bin/bash
#SBATCH --time=24:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=100G
#SBATCH --job-name=humann_SampleID

export OMP_NUM_THREADS=4
conda init bash
export CONDA3PATH=/mnt/home/shenyike/anaconda3
module load Conda/3
conda activate biobakery3

cd /$path_to_host_removed_reads

humann_config --update database_folders nucleotide /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/chocophlan

humann_config --update database_folders protein /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/uniref

humann_config --update database_folders utility_mapping /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/utility_mapping

humann -i SampleID_dehost.fastq.gz -o humann_results/humann_SampleID --threads 4

conda deactivate

scontrol show job $SLURM_JOB_ID

In [ ]:
#Use the below loop to generate #N batch jobs with different Sample ID and submit all jobs same time
#Humann runs more than 12 hours per sample, request enough run time.

In [ ]:
import pandas as pd
import numpy as np

name_index = pd.read_excel('File_Contain_Sample_List',header = None)
name_index = np.array(name_index[0])


s = ['#!/bin/bash','#SBATCH --time=24:00:00','#SBATCH --ntasks=1','#SBATCH --cpus-per-task=2',
     '#SBATCH --mem=100G']
for i in name_index:
    with open('sbatch_folderhumann/'+'humann_'+str(i)+'.sbatch','w') as file:
        for s_text in s:
            file.write(s_text)
            file.write('\n')
        file.write('#SBATCH --job-name=humann_'+str(i))
        file.write('\n')
        file.write('\n')
        file.write('export OMP_NUM_THREADS=4')
        file.write('\n')
        file.write('conda init bash')
        file.write('\n')
        file.write('export CONDA3PATH=/mnt/home/shenyike/anaconda3')
        file.write('\n')
        file.write('module load Conda/3')
        file.write('\n')
        file.write('conda activate biobakery3')
        file.write('\n')
        file.write('\n')
        file.write('cd $path_to_host_removed_reads')
        file.write('\n')
        file.write('\n')
        cmd1='humann_config --update database_folders nucleotide /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/chocophlan'
        file.write(cmd1)
        file.write('\n')
        file.write('\n')
        cmd2='humann_config --update database_folders protein /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/uniref'
        file.write(cmd2)
        file.write('\n')
        file.write('\n')
        cmd3='humann_config --update database_folders utility_mapping /mnt/home/shenyike/.local/lib/python3.6/site-packages/metaphlan/metaphlan_databases/utility_mapping'
        file.write(cmd3)
        file.write('\n')
        file.write('\n')
        cmd4= 'humann -i '+str(i)+'_dehost.fastq.gz'+' -o humann_results/humann_'+str(i)+' --threads 4'
        file.write(cmd4)
        file.write('\n')
        file.write('\n')
        file.write('conda deactivate')
        file.write('\n')
        file.write('\n')
        file.write('scontrol show job $SLURM_JOB_ID')

#### Now we have obtained pathway and gene-family table as main output; MetaPhlAn table in the intermediate output.
#### Transform them to relative abundance following biobakery instructions for downstream analysis. 
#### Please install all dependencies following developers instructions:
- FastQC and multiQC
- Trimmomatic
- Bowtie2
- Humann3
- Conda

### Please refer to R scripts for downstream analysis. 